# DicoCo

Alban Zurkowski

In [1]:
# import necessaire à la création des widgets
from IPython.display import display, clear_output, HTML, Markdown, Audio
import ipywidgets as widgets
from requests_html import HTMLSession
from bs4 import BeautifulSoup
import pandas as pd
import os
import re

os.chdir(r"D:\Cours\annee3\Prononciation\cnam-athon-pronunciation-main\data\dictionary")

In [2]:
buchanan = pd.read_csv("Buchanan_enrichi.csv", sep=",", encoding="utf-8")

In [32]:
#récupérer les URL des fichiers MP3
def getMP3URL(mot):
    session = HTMLSession()
    response = session.get("https://dictionary.cambridge.org/fr/dictionnaire/anglais/"+mot)
    html_doc = response.content
    soup = BeautifulSoup(html_doc, 'html.parser')
    
    #on prend que les fichiers mp3
    mp3all = soup.findAll(type="audio/mpeg")
    
    #on ne prend que le premier audio américain et anglais
    uk_found = False
    us_found = False
    for m in mp3all:
        if "uk_pron" in m.get_attribute_list('src')[0] and uk_found == False:
            uk_pron = "https://dictionary.cambridge.org"+m.get_attribute_list('src')[0]
            uk_found = True
        elif "us_pron" in m.get_attribute_list('src')[0] and us_found == False:
            us_pron = "https://dictionary.cambridge.org"+m.get_attribute_list('src')[0]
            us_found = True

    return uk_pron, us_pron

#récupérer un IPA du mot sur le site Cambridge et son "part of speech"
def getIPAPOS(mot):
    session = HTMLSession()
    response = session.get("https://dictionary.cambridge.org/fr/dictionnaire/anglais/"+mot)
    html_doc = response.content
    soup = BeautifulSoup(html_doc, 'html.parser')
    #on récupère la prononciation IPA et on ne renverra que le premier élément
    IPA = soup.findAll("span", {"class": "ipa dipa lpr-2 lpl-1"})
    typeMot = soup.findAll ("span", {"class": "pos dpos"})
    
    POS = []
    ipacontent = []
    
    for motIPA in IPA:
        ipacontent.append(motIPA.text)
        
    for categoryMot in typeMot:
        POS.append(categoryMot.text)
        
        
    return ipacontent[0], POS[0]

In [33]:
# definition du widget permettant de taper un texte 
textTarget = widgets.Text(
       value="",
       description='Entrer un mot',
       layout={'width': '200px'} )

# definition du widget permettant de taper un texte pour l'ipa
ipaTarget = widgets.Text(
       value="",
       description='IPA',
       layout={'width': '200px'} )

# sélection d'une plage de valeurs
rangeSyllMin = 1
rangeSyllMax = 27
optionsNbSyll = range(1, 27)
wid_nbsyll = widgets.SelectionRangeSlider(
    options=optionsNbSyll,
    index=(min(optionsNbSyll)-1,max(optionsNbSyll)-1),
    description='Nb syllabes',
    disabled=False
)

nbResultMax = 20
wid_nbResult = widgets.IntSlider(
    value=nbResultMax,
    min=1,
    max=nbResultMax,
    step=1,
    description='Nb résultats max.',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

In [37]:
# définition du widget button permettant de lancer l'extraction
button = widgets.Button(description='Démarrer')

# définition du widget permettant l'affichage d'une sortie
cambridge = widgets.Output()
longman = widgets.Output()
linguee = widgets.Output()
tableau = widgets.Output()
cam_audio_uk = widgets.Output()
cam_audio_us = widgets.Output()

def removeRegex(inputString):
    outputstring = ""
    for c in inputString:
        if c.isalpha():
            outputstring = outputstring + c
    return outputstring

# spécifier les actions à réaliser lors de l'appui sur le bouton
def on_button_clicked(_):
    
    # récupérer le texte saisi par l'utilisateur
    if len(textTarget.value) != 0:
        colToUse = 'transcription'
        targetWord = removeRegex(textTarget.value)
        ipaTarget.value
        ipaTarget.value = getIPAPOS(targetWord)[0]
        
    else:
        colToUse = 'ipa'
        targetWord = ipaTarget.value
        
    print("POS:",getIPAPOS(targetWord)[1])
    
    with cambridge:
        clear_output()
        urlCambridge = "https://dictionary.cambridge.org/fr/dictionnaire/anglais/"
        urlTarget = urlCambridge+targetWord
        display(HTML('<a href="'+urlTarget+'" target="_blank" >'+'Cambridge : '+targetWord+'</a>'))

    with longman:
        clear_output()        
        urlLongman = "https://www.ldoceonline.com/dictionary/"
        urlTarget = urlLongman+targetWord
        display(HTML('<a href="'+urlTarget+'" target="_blank" >'+'longman : '+targetWord+'</a>'))

    with linguee:
        clear_output()
        urlLinguee = "https://www.linguee.fr/anglais-francais/traduction/"
        urlTarget = urlLinguee+targetWord+'.html'
        display(HTML('<a href="'+urlTarget+'" target="_blank" >'+'linguee : '+targetWord+'</a>'))
        
    with cam_audio_uk:
        if colToUse == 'transcription':
            clear_output()        
            display(Audio(url=getMP3URL(targetWord)[0]))
        
    with cam_audio_us:
        if colToUse == 'transcription':
            clear_output()        
            display(Audio(url=getMP3URL(targetWord)[1]))
        
    #tableau filtré
    buchananFiltered = buchanan[buchanan[colToUse].str.contains(textTarget.value, na=False, regex=True)]
    buchananFiltered = buchananFiltered[buchananFiltered['sizePron'].between(wid_nbsyll.value[0], wid_nbsyll.value[1], inclusive=True)]


    with tableau:
        clear_output()
        display(buchananFiltered.head(n=wid_nbResult.value))
        
button.on_click(on_button_clicked)

In [38]:
# définition du widget button permettant de lancer l'extraction
button = widgets.Button(description='Démarrer')

# définition du widget permettant l'affichage d'une sortie
out1 = widgets.Output()
out2 = widgets.Output()
button.on_click(on_button_clicked)

Partie pris de ne pas générer les audios de tous les mots car beaucoup trop à charger (au moins sur mon PC).

In [39]:
# les widgets de filtre input
vbFiltres = widgets.HBox([textTarget, ipaTarget, wid_nbsyll, wid_nbResult, button])
# les widgets d'output
vbOuts = widgets.VBox([cambridge,longman,linguee, tableau])

#buchananFiltered = buchanan[buchanan['transcription'].str.contains(textTarget.value, na=False, regex=True)]
#buchananFiltered = buchananFiltered[buchananFiltered['sizePron'].between(wid_nbsyll.value[0], wid_nbsyll.value[1], inclusive=True)]
vAudios = widgets.HBox([widgets.VBox([widgets.Label(value="UK"),cam_audio_uk]), widgets.VBox([widgets.Label(value="US"), cam_audio_us])])

hbAllWidgets = widgets.VBox([vbFiltres,vbOuts, vAudios])
hbAllWidgets

POS: noun
